## Data preparation

In [1]:
import io
import os
import re
import shutil
import string
import tensorflow as tf
import numpy as np

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers import TextVectorization

In [ ]:
url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

dataset = tf.keras.utils.get_file("aclImdb_v1.tar.gz", url,
                                  untar=True, cache_dir='.',
                                  cache_subdir='')

dataset_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')
os.listdir(dataset_dir)

In [ ]:
train_dir = os.path.join(dataset_dir, 'train')
os.listdir(train_dir)

In [ ]:
remove_dir = os.path.join(train_dir, 'unsup')
shutil.rmtree(remove_dir)

In [2]:
batch_size = 16
seed = 12345
train_ds = tf.keras.utils.text_dataset_from_directory(
                            'aclImdb/train', batch_size=batch_size, 
                            validation_split=0.2,
                            subset='training', seed=seed)
val_ds = tf.keras.utils.text_dataset_from_directory(
                            'aclImdb/train', batch_size=batch_size, 
                            validation_split=0.2,
                            subset='validation', seed=seed)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.


2023-02-17 10:00:25.520581: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-17 10:00:25.525872: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-17 10:00:25.526147: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-17 10:00:25.526641: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorF

Found 25000 files belonging to 2 classes.
Using 5000 files for validation.


In [3]:
for x, y in train_ds:
    print(x.shape)
    print(y.shape)
    break

(16,)
(16,)


In [2]:
vocab_size   = 20000
sequence_len = 200

def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, "<br />", " ")
    return tf.strings.regex_replace(
        stripped_html, f"[{re.escape(string.punctuation)}]", ""
    )

vectorization = tf.keras.layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_len,
)

vectorization.adapt(train_ds.map(lambda text, label: text))

NameError: name 'train_ds' is not defined

In [5]:
def vectorize_text(text, label):
    text = tf.expand_dims(text, -1)
    return vectorization(text), label

train_ds = train_ds.map(vectorize_text)
val_ds = val_ds.map(vectorize_text)

In [6]:
for text_batch, label_batch in train_ds:
    print(label_batch[0].numpy())
    print(text_batch.numpy()[0])
    break

1
[   11    17     7 11563    28     5    55  1567    91     8    29   239
     2  4102   192  6516     3  5937     1   101     7    33 18665     6
     1     3     2 10786  2660  2875  7038     3  8241     9  4209  1288
    38  4588  3534   689  8878     3  1636    38   167    96    25   224
     3   463   591    12    10    68  2094   232     2   333     5     2
     1     8     2   540  4050     2   711     7  6453   185     6     2
  2782   563   160    45     3  1272   968     2 12119    82     4  4724
   112   201     3   345   203     1   826  4746     3  2039   134    12
    24   383   856     8  3002     1     5     2   521     8   643     6
   386     2   307  4401    57   136   134    24   965    41   856     2
  3513     7     1     3  1151     6     2 12119     1     7    33   662
  7840     8     4   521     8     1     2     2   839     7   383  3107
   138     2     1     5     2   307    57    11     7    21   698 17669
     4   905     5 10534    15     2   223     7 

In [7]:
train_ds = train_ds.cache().prefetch(buffer_size=10)
val_ds = val_ds.cache().prefetch(buffer_size=10)

## Training

In [3]:
from tensorflow.keras import layers

class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super().__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        return self.layernorm2(out1 + ffn_output)

In [4]:
# Two seperate embedding layers, one for tokens, one for token index (positions)

class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super().__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

In [10]:
from tensorflow import keras

embed_dim = 128  # Embedding size for each token
num_heads = 6    # Number of attention heads
ff_dim = 128     # Hidden layer size in feed forward network inside transformer

inputs = layers.Input(shape=(sequence_len,))
embedding_layer = TokenAndPositionEmbedding(sequence_len, vocab_size, embed_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = layers.Flatten()(x)
x = layers.Dense(128, activation="relu")(x)
outputs = layers.Dense(2, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 200)]             0         
                                                                 
 token_and_position_embeddin  (None, 200, 128)         2585600   
 g (TokenAndPositionEmbeddin                                     
 g)                                                              
                                                                 
 transformer_block (Transfor  (None, 200, 128)         429184    
 merBlock)                                                       
                                                                 
 flatten (Flatten)           (None, 25600)             0         
                                                                 
 dense_2 (Dense)             (None, 128)               3276928   
                                                             

In [11]:
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
history = model.fit(train_ds, batch_size=32, epochs=5, validation_data=val_ds)

Epoch 1/5


2023-02-17 10:00:32.975000: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-02-17 10:00:32.978656: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7f6536a1d2a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-02-17 10:00:32.978667: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce RTX 3070, Compute Capability 8.6
2023-02-17 10:00:32.981539: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-02-17 10:00:33.073006: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1250/1250 [==============================] - 43s 32ms/step - loss: 1.2075 - accuracy: 0.5011 - val_loss: 0.7026 - val_accuracy: 0.5004
Epoch 2/5
1250/1250 [==============================] - 9s 7ms/step - loss: 0.4030 - accuracy: 0.8122 - val_loss: 0.4045 - val_accuracy: 0.8408
Epoch 3/5
1250/1250 [==============================] - 9s 7ms/step - loss: 0.1080 - accuracy: 0.9602 - val_loss: 0.4618 - val_accuracy: 0.8466
Epoch 4/5
1250/1250 [==============================] - 8s 7ms/step - loss: 0.0518 - accuracy: 0.9826 - val_loss: 0.6062 - val_accuracy: 0.8512
Epoch 5/5
1250/1250 [==============================] - 8s 7ms/step - loss: 0.0262 - accuracy: 0.9909 - val_loss: 1.0132 - val_accuracy: 0.8080


In [5]:
from tensorflow import keras

embed_dim = 128  # Embedding size for each token
num_heads = 6    # Number of attention heads
ff_dim = 128     # Hidden layer size in feed forward network inside transformer

inputs = layers.Input(shape=(sequence_len,))
embedding_layer = TokenAndPositionEmbedding(sequence_len, vocab_size, embed_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dense(128, activation="relu")(x)
outputs = layers.Dense(2, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 200)]             0         
                                                                 
 token_and_position_embeddin  (None, 200, 128)         2585600   
 g (TokenAndPositionEmbeddin                                     
 g)                                                              
                                                                 
 transformer_block (Transfor  (None, 200, 128)         429184    
 merBlock)                                                       
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense_2 (Dense)             (None, 128)               16512 